In [1]:
import cv2
import numpy as np 
import os 
from random import shuffle
from tqdm import tqdm 
from scipy import misc 
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
import tensorflow as tf
import glob
import matplotlib.pyplot as plt 
import dlib 
from skimage import io
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
tf.compat.v1.reset_default_graph()
print(os.getcwd())
TRAIN_DIR = 'C:/Users/ctgfa/images/train/B'
TEST_DIR = 'C:/Users/ctgfa/images/test/B'
IMG_SIZE = 200
boxScale = 1
LR = 1e-3 
MODEL_NAME = 'qrecognition.model'.format(LR,'2conv-basic')


C:\Users\ctgfa


In [3]:
#Declaring the necessary functions 

def label_img(img):
    word = img.split('(')[-1]
    word_label = word[0]
    print(word_label)
    if word_label == 'B':
        return [1,0]
    elif word_label == 'S':
        return [0,1]

def detect_faces(image):
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image,1)
    faces_frames = [(x.left(),x.top(),x.right(),x.bottom()) for x in detected_faces]
    return faces_frames


def create_train_data():
    training_data =[]
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        print(path)
        img = plt.imread(path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        detected_faces = detect_faces(img)
        for n,face_rect in enumerate(detected_faces):
            img = Image.fromarray(img).crop(face_rect)
            img = np.array(img)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy',training_data)
    return training_data
                
        
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        imgnum = img.split('.')[-2]
        img_num = get_num(imgnum)
        img = plt.imread(path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        detected_faces = detect_faces(img)
        for n, face_rect in enumerate (detected_faces):
            img = Image.fromarray(img).crop(face_rect)
            img = np.array(img)
        img = cv2.resize(img,(IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img),img_num])
        

In [5]:
train_data = create_train_data()
train = train_data[:-2]
test = train_data[-2:]
X= np.array([i[0] for i in train]).reshape(-1,200,200,1)
Y= [i[1] for i in train]
test_x =  np.array([i[0] for i in test]).reshape(-1,200,200,1)
test_y = [i[1] for i in test]
convnet = input_data(shape = [None,200,200,1], name ='input')

convnet =conv_2d(convnet,4,5,activation = 'relu')
convnet = max_pool_2d(convnet,5)

convnet = conv_2d(convnet,5,5,activation = 'relu')
convnet = max_pool_2d(convnet,5)

convnet = conv_2d(convnet,8,5,activation = 'relu')
convnet = max_pool_2d(convnet,5)

convnet = fully_connected(convnet,8,activation = 'relu')
convnet = dropout(convnet,0.2)

convnet = fully_connected(convnet, 2, activation = 'softmax')
convnet = regression(convnet,optimizer= 'adam', learning_rate = LR, loss = 'categorical_crossentropy', name = 'targets')
model = tflearn.DNN(convnet, tensorboard_dir='log')
model.fit({'input':X},{'targets':Y}, n_epoch = 1,validation_set = ({'input':test_x},{'targets':test_y}),snapshot_step= 500,show_metric = True,run_id = MODEL_NAME)


  0%|          | 0/4 [00:00<?, ?it/s]

1
C:/Users/ctgfa/images/train/B\A(1).jpeg


 25%|██▌       | 1/4 [00:00<00:01,  2.45it/s]

2
C:/Users/ctgfa/images/train/B\A(2).jpeg


 50%|█████     | 2/4 [00:00<00:00,  2.92it/s]

...

 75%|███████▌  | 3/4 [00:01<00:00,  2.91it/s]

W
C:/Users/ctgfa/images/train/B\WhatsApp Image 2022-12-15 at 19.49.03.jpeg


100%|██████████| 4/4 [00:01<00:00,  2.93it/s]


---------------------------------
Run id: qrecognition.model
Log directory: log/
INFO:tensorflow:Summary name Accuracy/Adam_0 (raw) is illegal; using Accuracy/Adam_0__raw_ instead.
INFO:tensorflow:Summary name Accuracy_1/Adam_1 (raw) is illegal; using Accuracy_1/Adam_1__raw_ instead.
---------------------------------
Training samples: 4
Validation samples: 4
--


TypeError: float() argument must be a string or a number, not 'NoneType'

In [ ]:
test_data = process_test_data()
fig = plt.figure()
for num,data in enumerate(test_data[-2:]):
    img_num = data[1]
    img_data = data[0]
    y = fig.add_subplot(3,4,num+1)
    orig = img_data
    data = img_data.reshape(IMG_SIZE,IMG_SIZE,1)
    model_out = model.predict([data])[0]
    if np.argmax(model_out)==0: str_label = "Sajjad Salam"
    elif np.argmax(model_out)==1: str_label = "Abdul Basset"
    y.imshow(orig,cmap = 'gray')
    plt.title(str_label)
    y.axes.get_xaxis().set_visible(False)
    y.axes.get_yaxis().set_visible(False)
plt.show()

    
    